In [16]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [17]:
def prepare_data(path):
    df = pd.read_csv(path)
    df = df.drop(labels = ['Host','Vcs','Project','File','PL','IssueTracking'],axis=1)
    df = df.dropna()
    df = df[['TLOC', 'TNF', 'TNC', 'TND', 'LOC', 'CL', 'NStmt', 'NFunc',
       'RCC', 'MNL', 'avg_WMC', 'max_WMC', 'total_WMC', 'avg_DIT', 'max_DIT',
       'total_DIT', 'avg_RFC', 'max_RFC', 'total_RFC', 'avg_NOC', 'max_NOC',
       'total_NOC', 'avg_CBO', 'max_CBO', 'total_CBO', 'avg_DIT.1',
       'max_DIT.1', 'total_DIT.1', 'avg_NIV', 'max_NIV', 'total_NIV',
       'avg_NIM', 'max_NIM', 'total_NIM', 'avg_NOM', 'max_NOM', 'total_NOM',
       'avg_NPBM', 'max_NPBM', 'total_NPBM', 'avg_NPM', 'max_NPM', 'total_NPM',
       'avg_NPRM', 'max_NPRM', 'total_NPRM', 'avg_CC', 'max_CC', 'total_CC',
       'avg_FANIN', 'max_FANIN', 'total_FANIN', 'avg_FANOUT', 'max_FANOUT',
       'total_FANOUT', 'NRev', 'NFix', 'avg_AddedLOC', 'max_AddedLOC',
       'total_AddedLOC', 'avg_DeletedLOC', 'max_DeletedLOC',
       'total_DeletedLOC', 'avg_ModifiedLOC', 'max_ModifiedLOC',
       'total_ModifiedLOC','Buggy']]
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.Buggy.values
    X = df.drop(labels = ['Buggy'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Buggy')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def load_data(path,target):
    df = pd.read_csv(path)
    if path == 'data/jm1.csv':
        df = df[~df.uniq_Op.str.contains("\?")]
    y = df[target]
    X = df.drop(labels = target, axis = 1)
    X = X.apply(pd.to_numeric)
    return X,y

# Cluster Driver
def cluster_driver(df,print_tree = True):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
    #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
    #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

In [18]:
def get_predicted(cluster_data_loc,results_loc,fold,data_location):
    train_data = pd.read_pickle(cluster_data_loc + '/train_data.pkl')
    train_projects = train_data.index.values.tolist()
    cluster,cluster_tree,max_depth = cluster_driver(train_data)
    test_data = pd.read_pickle(cluster_data_loc + '/test_data.pkl')
    test_projects = test_data.index.values.tolist()
    goals = ['recall','precision','pf']
    for goal in goals:
        goal_df = pd.read_csv(results_loc + '/1385_LR_bellwether_' + goal + '.csv')
        goal_df.rename(columns = {'Unnamed: 0':'s_project'},inplace = True)
        goal_df = goal_df[goal_df['s_project'].isin(train_projects)]
        goal_df = goal_df[train_projects]
        if not Path(data_location).is_dir():
            os.makedirs(Path(data_location))
        print(data_location)
        goal_df.to_csv(data_location + '/bellwether_0_' + goal + '.csv')
        

In [20]:
for i in range(20):
    fold = str(i)
    data_location = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_' + fold
    cluster_data_loc = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/2/fold_' + fold
    results_loc = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0'
    results = get_predicted(cluster_data_loc,results_loc,fold,data_location)
    
    

[cluster_id=0] N_children: 7 N_samples: 627
> [cluster_id=1] N_children: 0 N_samples: 1
> [cluster_id=2] N_children: 0 N_samples: 1
> [cluster_id=3] N_children: 5 N_samples: 55
> > [cluster_id=4] N_children: 0 N_samples: 9
> > [cluster_id=5] N_children: 0 N_samples: 9
> > [cluster_id=6] N_children: 0 N_samples: 7
> > [cluster_id=7] N_children: 0 N_samples: 17
> > [cluster_id=8] N_children: 0 N_samples: 13
> [cluster_id=9] N_children: 10 N_samples: 127
> > [cluster_id=10] N_children: 0 N_samples: 3
> > [cluster_id=11] N_children: 0 N_samples: 9
> > [cluster_id=12] N_children: 0 N_samples: 19
> > [cluster_id=13] N_children: 0 N_samples: 15
> > [cluster_id=14] N_children: 0 N_samples: 4
> > [cluster_id=15] N_children: 0 N_samples: 17
> > [cluster_id=16] N_children: 0 N_samples: 15
> > [cluster_id=17] N_children: 0 N_samples: 19
> > [cluster_id=18] N_children: 0 N_samples: 11
> > [cluster_id=19] N_children: 0 N_samples: 15
> [cluster_id=20] N_children: 16 N_samples: 183
> > [cluster_id=21]

/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_2
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_2
[cluster_id=0] N_children: 8 N_samples: 627
> [cluster_id=1] N_children: 7 N_samples: 79
> > [cluster_id=2] N_children: 0 N_samples: 2
> > [cluster_id=3] N_children: 0 N_samples: 10
> > [cluster_id=4] N_children: 0 N_samples: 13
> > [cluster_id=5] N_children: 0 N_samples: 14
> > [cluster_id=6] N_children: 0 N_samples: 20
> > [cluster_id=7] N_children: 0 N_samples: 12
> > [cluster_id=8] N_children: 0 N_samples: 8
> [cluster_id=9] N_children: 2 N_samples: 7
> > [cluster_id=10] N_children: 0 N_samples: 1
> > [cluster_id=11] N_children: 0 N_samples: 6
> [cluster_id=12] N_children: 2 N_samples: 15
> > [cluster_id=13] N_children: 0 N_samples: 10
> > [cluster_id=14] N_children: 0 N_samples: 5
> [cluster_id=15] N_children: 3 N_samples: 33
> > [cluster_id=16] N_children: 0 N_samples: 18
>

/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_5
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_5
[cluster_id=0] N_children: 9 N_samples: 627
> [cluster_id=1] N_children: 7 N_samples: 81
> > [cluster_id=2] N_children: 0 N_samples: 2
> > [cluster_id=3] N_children: 0 N_samples: 7
> > [cluster_id=4] N_children: 0 N_samples: 17
> > [cluster_id=5] N_children: 0 N_samples: 10
> > [cluster_id=6] N_children: 0 N_samples: 18
> > [cluster_id=7] N_children: 0 N_samples: 16
> > [cluster_id=8] N_children: 0 N_samples: 11
> [cluster_id=9] N_children: 2 N_samples: 4
> > [cluster_id=10] N_children: 0 N_samples: 1
> > [cluster_id=11] N_children: 0 N_samples: 3
> [cluster_id=12] N_children: 10 N_samples: 88
> > [cluster_id=13] N_children: 0 N_samples: 3
> > [cluster_id=14] N_children: 0 N_samples: 17
> > [cluster_id=15] N_children: 0 N_samples: 14
> > [cluster_id=16] N_children: 0 N_samples: 8

/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_8
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_8
[cluster_id=0] N_children: 9 N_samples: 628
> [cluster_id=1] N_children: 8 N_samples: 79
> > [cluster_id=2] N_children: 0 N_samples: 2
> > [cluster_id=3] N_children: 0 N_samples: 6
> > [cluster_id=4] N_children: 0 N_samples: 16
> > [cluster_id=5] N_children: 0 N_samples: 5
> > [cluster_id=6] N_children: 0 N_samples: 20
> > [cluster_id=7] N_children: 0 N_samples: 13
> > [cluster_id=8] N_children: 0 N_samples: 11
> > [cluster_id=9] N_children: 0 N_samples: 6
> [cluster_id=10] N_children: 2 N_samples: 3
> > [cluster_id=11] N_children: 0 N_samples: 1
> > [cluster_id=12] N_children: 0 N_samples: 2
> [cluster_id=13] N_children: 9 N_samples: 91
> > [cluster_id=14] N_children: 0 N_samples: 3
> > [cluster_id=15] N_children: 0 N_samples: 20
> > [cluster_id=16] N_children: 0 N_samples: 13
>

/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_11
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_11
[cluster_id=0] N_children: 8 N_samples: 627
> [cluster_id=1] N_children: 9 N_samples: 111
> > [cluster_id=2] N_children: 0 N_samples: 1
> > [cluster_id=3] N_children: 0 N_samples: 10
> > [cluster_id=4] N_children: 0 N_samples: 16
> > [cluster_id=5] N_children: 0 N_samples: 11
> > [cluster_id=6] N_children: 0 N_samples: 15
> > [cluster_id=7] N_children: 0 N_samples: 18
> > [cluster_id=8] N_children: 0 N_samples: 20
> > [cluster_id=9] N_children: 0 N_samples: 10
> > [cluster_id=10] N_children: 0 N_samples: 10
> [cluster_id=11] N_children: 0 N_samples: 1
> [cluster_id=12] N_children: 2 N_samples: 8
> > [cluster_id=13] N_children: 0 N_samples: 6
> > [cluster_id=14] N_children: 0 N_samples: 2
> [cluster_id=15] N_children: 13 N_samples: 130
> > [cluster_id=16] N_children: 0 N_samples

/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_14
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_14
[cluster_id=0] N_children: 9 N_samples: 627
> [cluster_id=1] N_children: 8 N_samples: 85
> > [cluster_id=2] N_children: 0 N_samples: 2
> > [cluster_id=3] N_children: 0 N_samples: 7
> > [cluster_id=4] N_children: 0 N_samples: 19
> > [cluster_id=5] N_children: 0 N_samples: 6
> > [cluster_id=6] N_children: 0 N_samples: 19
> > [cluster_id=7] N_children: 0 N_samples: 15
> > [cluster_id=8] N_children: 0 N_samples: 12
> > [cluster_id=9] N_children: 0 N_samples: 5
> [cluster_id=10] N_children: 2 N_samples: 4
> > [cluster_id=11] N_children: 0 N_samples: 1
> > [cluster_id=12] N_children: 0 N_samples: 3
> [cluster_id=13] N_children: 10 N_samples: 88
> > [cluster_id=14] N_children: 0 N_samples: 3
> > [cluster_id=15] N_children: 0 N_samples: 13
> > [cluster_id=16] N_children: 0 N_samples: 1

/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_17
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_17
[cluster_id=0] N_children: 9 N_samples: 628
> [cluster_id=1] N_children: 7 N_samples: 74
> > [cluster_id=2] N_children: 0 N_samples: 2
> > [cluster_id=3] N_children: 0 N_samples: 7
> > [cluster_id=4] N_children: 0 N_samples: 18
> > [cluster_id=5] N_children: 0 N_samples: 6
> > [cluster_id=6] N_children: 0 N_samples: 19
> > [cluster_id=7] N_children: 0 N_samples: 9
> > [cluster_id=8] N_children: 0 N_samples: 13
> [cluster_id=9] N_children: 2 N_samples: 5
> > [cluster_id=10] N_children: 0 N_samples: 1
> > [cluster_id=11] N_children: 0 N_samples: 4
> [cluster_id=12] N_children: 9 N_samples: 94
> > [cluster_id=13] N_children: 0 N_samples: 3
> > [cluster_id=14] N_children: 0 N_samples: 20
> > [cluster_id=15] N_children: 0 N_samples: 14
> > [cluster_id=16] N_children: 0 N_samples: 9
